In [14]:
from math import sqrt
import polars as pl
from plotly import express as px

In [15]:
audit = (
    # pl.read_ndjson("../generated/dev5_ACSEmployment_binarized.jsonl")
    pl.read_ndjson("../generated/dev5_ACSEmployment_binarized_linear.jsonl")
    # .select(pl.all().exclude("model_params"))
    .group_by(
        "dataset",
        "model_name",
        "model_params",
        "strategy",
        "strategy_params",
        "audit_budget",
        "detection_tpr",
        "detection_tnr",
    )
    .mean()
    .with_columns(
        strategy_json=pl.col("strategy_params").struct.json_encode(),
        model_json=pl.col("model_params").struct.json_encode(),
    )
    .sort("detection_tpr", "detection_tnr", descending=[True, False])
)

In [16]:
fig = px.scatter(
    audit.filter(
        pl.col("detection_tnr") == 1.0,
        pl.col("detection_tpr") == 1.0,
        pl.col("strategy") == "honest",
    ),
    x="demographic_parity_user",
    y="utility_user",
    color="model_name",
    category_orders={"strategy": audit["strategy"].unique().sort().to_list()},
    height=600,
    width=800,
    title="Honest platform",
)
fig.show()

In [17]:
fig = px.scatter(
    audit.filter(
        pl.col("detection_tnr") == 1.0,
        pl.col("detection_tpr") == 0.5,
    ).with_columns(
        is_honest=pl.when(pl.col("strategy") == "honest")
        .then("strategy")
        .otherwise(pl.lit("sneaky")),
        size=10,
    ),
    x="demographic_parity_audit",
    y="utility_user",
    color="strategy",
    symbol="is_honest",
    size="size",
    size_max=10,
    hover_data=["strategy_json", "model_json", "model_name"],
    category_orders={"strategy": audit["strategy"].unique().sort().to_list()},
    height=600,
    width=800,
    title="Sneaky platform, detection tpr=50%",
)
fig.show()

In [18]:
fig = px.scatter(
    audit.with_columns(
        is_honest=pl.when(pl.col("strategy") == "honest")
        .then("strategy")
        .otherwise(pl.lit("sneaky")),
        size=10,
    ),
    x="demographic_parity_audit",
    y="utility_user",
    color="strategy",
    symbol="is_honest",
    size="size",
    size_max=10,
    facet_row="detection_tpr",
    facet_col="detection_tnr",
    hover_data=["strategy_json", "model_json", "model_name"],
    height=1_000,
    width=1_300,
)

# fig.update_xaxes(range=[0, 0.1])
# fig.update_yaxes(range=[0.81, 0.83])
fig.show()

# What is the best lying strategy ?

In [19]:
manipulation = (
    pl.read_ndjson("../generated/manipulation5_ACSEmployment_binarized_skrub.jsonl")
    .with_columns(
        min_conditional_accuracy=pl.col("conditional_accuracy_user").list.min(),
        strategy_instance=pl.concat_str(
            pl.col("strategy"), pl.col("strategy_params").struct.json_encode()
        ),
        strategy_json=pl.col("strategy_params").struct.json_encode(),
    )
    .group_by(
        "dataset",
        "model_name",
        # "model_params",
        "strategy",
        "strategy_params",
        "strategy_json",
        "audit_budget",
        "detection_tpr",
        "detection_tnr",
    )
    .agg(pl.col("*").mean(), pl.col("*").std().name.suffix("_std"))
    .sort("detection_tpr", "detection_tnr", descending=[True, False])
)

In [20]:
manipulation

dataset,model_name,strategy,strategy_params,strategy_json,audit_budget,detection_tpr,detection_tnr,model_params,entropy,fit_time,utility_user,utility_audit,conditional_accuracy_user,demographic_parity_user,demographic_parity_audit,min_conditional_accuracy,strategy_instance,model_params_std,entropy_std,fit_time_std,utility_user_std,utility_audit_std,conditional_accuracy_user_std,demographic_parity_user_std,demographic_parity_audit_std,min_conditional_accuracy_std,strategy_instance_std
str,str,str,struct[2],str,i64,f64,f64,struct[1],f64,f64,f64,f64,list[f64],f64,f64,f64,str,struct[1],f64,f64,f64,f64,list[f64],f64,f64,f64,str
"""ACSEmployment""","""unconstrained""","""randomized_response""","{7.0,null}","""{""epsilon"":7.0,""theta"":null}""",1000,1.0,1.0,null,2.3527e9,0.307695,0.85047,0.8578,null,0.075586,0.085552,0.827184,null,null,9.0409e8,0.037479,0.003506,0.00991,null,0.007315,0.033986,0.026051,null
"""ACSEmployment""","""unconstrained""","""randomized_response""","{2.0,null}","""{""epsilon"":2.0,""theta"":null}""",1000,1.0,1.0,null,2.3527e9,0.31657,0.850761,0.8554,null,0.071064,0.082495,0.820069,null,null,9.0409e8,0.043356,0.002691,0.010714,null,0.009637,0.056131,0.033059,null
"""ACSEmployment""","""unconstrained""","""honest""","{null,null}","""{""epsilon"":null,""theta"":null}""",1000,1.0,1.0,null,2.3527e9,0.282617,0.850965,0.8594,null,0.070556,0.07326,0.828728,null,null,9.0409e8,0.064581,0.002599,0.010877,null,0.010239,0.047719,0.025385,null
"""ACSEmployment""","""unconstrained""","""randomized_response""","{5.0,null}","""{""epsilon"":5.0,""theta"":null}""",1000,1.0,1.0,null,2.3527e9,0.300273,0.850572,0.8568,null,0.074223,0.08138,0.829126,null,null,9.0409e8,0.034879,0.00303,0.009203,null,0.007434,0.048795,0.024369,null
"""ACSEmployment""","""unconstrained""","""ROC_mitigation""","{null,0.533333}","""{""epsilon"":null,""theta"":0.5333…",1000,1.0,1.0,null,2.3527e9,0.283449,0.849916,0.857,null,0.043705,0.047089,0.82529,null,null,9.0409e8,0.046859,0.003651,0.011113,null,0.008237,0.054707,0.027566,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ACSEmployment""","""unconstrained""","""randomized_response""","{7.0,null}","""{""epsilon"":7.0,""theta"":null}""",1000,0.0,1.0,null,2.5082e9,0.247828,0.850297,0.854833,null,0.071154,0.071932,0.82186,null,null,8.9381e8,0.036834,0.003031,0.01172,null,0.011911,0.046286,0.030199,null
"""ACSEmployment""","""unconstrained""","""randomized_response""","{1.0,null}","""{""epsilon"":1.0,""theta"":null}""",1000,0.0,1.0,null,2.5082e9,0.317179,0.850079,0.856167,null,0.06932,0.077661,0.831715,null,null,8.9381e8,0.120047,0.002544,0.012057,null,0.007882,0.047571,0.025068,null
"""ACSEmployment""","""unconstrained""","""always_yes""","{null,null}","""{""epsilon"":null,""theta"":null}""",1000,0.0,1.0,null,2.3527e9,0.25048,0.851037,0.86,null,0.073415,0.080107,0.834951,null,null,9.0409e8,0.042649,0.002932,0.010677,null,0.006478,0.049504,0.021709,null


In [21]:
px.line(
    manipulation.sort("detection_tpr").with_columns(
        pl.col("min_conditional_accuracy_std") / sqrt(5)
    ),
    x="detection_tpr",
    y="min_conditional_accuracy",
    error_y="min_conditional_accuracy_std",
    facet_col="strategy",
    color="strategy_json",
)

In [22]:
px.line(
    manipulation.sort("detection_tpr").with_columns(
        pl.col("demographic_parity_audit_std") / sqrt(5)
    ),
    x="detection_tpr",
    y="demographic_parity_audit",
    error_y="demographic_parity_audit_std",
    facet_col="strategy",
    color="strategy_json",
)

In [23]:
px.line(
    manipulation.sort("detection_tpr").with_columns(
        pl.col("demographic_parity_user_std") / sqrt(5)
    ),
    x="detection_tpr",
    y="demographic_parity_user",
    error_y="demographic_parity_user_std",
    facet_col="strategy",
    color="strategy_json",
)

In [24]:
px.line(
    manipulation.sort("detection_tpr").with_columns(
        pl.col("utility_user_std") / sqrt(5)
    ),
    x="detection_tpr",
    y="utility_user",
    error_y="utility_user_std",
    facet_col="strategy",
    color="strategy_json",
)

# Finding datasets with the largest base rate difference

In [25]:
base_rate = pl.read_ndjson("../generated/disparities.jsonl")

In [26]:
px.line(
    base_rate,
    x="year",
    y="base_rate",
    color="state",
    facet_col="group",
    hover_data="len",
)